## Model Development

In [3]:
import pandas as pd
import skfuzzy as fuzz
import numpy as np
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

In [4]:
from ipynb.fs.full.weather_data_preprocess import data, train_data, test_data

In [3]:
def get_range(data):
    data_range = pd.DataFrame(columns = data.columns, index = ['max', 'min'])
    data_range.loc['max'] = [data[col].max() for col in data.columns]
    data_range.loc['min'] = [data[col].min() for col in data.columns]
    return data_range

range_df = get_range(data)

In [4]:
#Specifying the universe of discourse

universe = []
for col in range_df.columns:
        universe_col = np.arange(np.floor(range_df[col].min()), np.ceil(range_df[col].max()+1), 0.1)
        universe.append(universe_col)

In [5]:
# Median

def med(l, u):
    return np.median(np.arange(l, u, 0.1))

In [6]:
hum_lo  = fuzz.trimf(universe[0], [68, 68, med(68,83)])
hum_mid = fuzz.trimf(universe[0], [68, med(68,83), 84])
hum_hi  =  fuzz.trimf(universe[0], [med(68,83), 84, 84])

temp_lo  = fuzz.trimf(universe[1], [23, 23, med(23,26.1)])
temp_mid = fuzz.trimf(universe[1], [23, med(23,26.1), 26])
temp_hi  = fuzz.trimf(universe[1], [med(23,26.1), 26, 26])


heat_lo  = fuzz.trimf(universe[2], [23.3, 23.3, med(23.3,25.7)])
heat_mid = fuzz.trimf(universe[2], [23.3, med(23.3,25.7), 25.7])
heat_hi  = fuzz.trimf(universe[2], [med(23.3,25.7), 25.7, 25.7])

In [7]:
def fuzzify_hum(hum_val):
    hum_df = pd.DataFrame(hum_val)
    hum_df['low'] = fuzz.interp_membership(universe[0], hum_lo, hum_val)
    hum_df['mid'] = fuzz.interp_membership(universe[0], hum_mid, hum_val)
    hum_df['high'] = fuzz.interp_membership(universe[0], hum_hi, hum_val)

    hum_df['membership'] = hum_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
    hum_df['degree'] = hum_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)
    return hum_df

hum_val = train_data.iloc[:,0]
hum_df = fuzzify_hum(hum_val)

In [8]:
def fuzzify_temp(temp_val):
    temp_df = pd.DataFrame(temp_val)
    temp_df['low'] = fuzz.interp_membership(universe[1], temp_lo, temp_val)
    temp_df['mid'] = fuzz.interp_membership(universe[1], temp_mid, temp_val)
    temp_df['high'] = fuzz.interp_membership(universe[1], temp_hi, temp_val)

    temp_df['membership'] = temp_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
    temp_df['degree'] = temp_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)
    return temp_df
temp_val  = train_data.iloc[:,1]       
temp_df = fuzzify_temp(temp_val)

In [9]:
def fuzzify_heat(heat_val):
    heat_df = pd.DataFrame(heat_val)
    heat_df['low'] = fuzz.interp_membership(universe[2], heat_lo, heat_val)
    heat_df['mid'] = fuzz.interp_membership(universe[2], heat_mid,heat_val)
    heat_df['high'] = fuzz.interp_membership(universe[2],heat_hi, heat_val)

    heat_df['membership'] = heat_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
    heat_df['degree'] = heat_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)
    return heat_df
heat_val = train_data.iloc[:,2]
heat_df = fuzzify_heat(heat_val)

In [10]:
# Merge the membership columns from predictors and target to get an initial rule set

def merge_rule(df, hum_df, temp_df, heat_df):
    rule_df = df.copy()
    rule_df['humidity'] = hum_df['membership']
    rule_df['temperature'] = temp_df['membership']
    rule_df['heat'] = heat_df['membership']
    rule_df['degree'] = hum_df['degree']*temp_df['degree']*heat_df['degree']
    return rule_df
 
rule_df = merge_rule(train_data, hum_df, temp_df, heat_df)
rule_df;

In [11]:
#Using max to get final rule set

rule_fuzzy = rule_df.groupby(['humidity', 'temperature']).max()
rule_fuzzy = rule_fuzzy.reset_index()
rule_fuzzy_deg = rule_fuzzy.pop('degree')